In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import time
from numba import njit

# Коэффициенты задачи
D_0 = 1.0  # коэффициент диффузии для основного материала
D_1 = 1e-2  # коэффициент диффузии для включений
T_0 = 0  # начальная температура во всем объеме пластины

# Параметры источников
source_positions = np.array([[10, 40], [30, 10]])  # Массив координат источников
source_powers = np.array([100, 100])  # Массив мощностей источников
source_periods = np.array([37, 73])  # Массив периодов для каждого источника

# Координаты проб
probes = [(20, 20), (20, 40), (40, 40), (40, 20)]

# Коэффициенты диффузии
def create_diffusion_matrix(N):
    """
    Создает матрицу коэффициентов диффузии для области размером N x N.
    Основной материал имеет коэффициент диффузии D_0, включения - D_1.

    Parameters:
    - N: размерность матрицы (NxN)

    Returns:
    - D: матрица коэффициентов диффузии
    """
    D = np.ones((N, N)) * D_0
    center = N // 3
    size = N // 6
    D[center:center + size, center:center + size] = D_1
    return D

# Функция для моделирования теплопроводности с использованием Numba
@njit
def heat_diffusion_optimized(N, t_m, dt, D, source_positions, source_powers, source_periods, probes):
    """
    Моделирует распространение тепла в пластине, решая уравнение диффузии.

    Parameters:
    - N: размерность сетки
    - t_m: общее время моделирования
    - dt: шаг по времени
    - D: матрица коэффициентов диффузии
    - source_positions: массив координат источников тепла
    - source_powers: массив мощностей источников
    - source_periods: массив периодов источников
    - probes: список точек проб для замера температуры

    Returns:
    - T: конечное температурное поле
    - probe_temps: температуры в точках проб для каждого временного шага
    """
    T = np.zeros((N, N))
    dx = 1.0
    probe_temps = np.zeros((len(probes), int(t_m / dt)))
    time_steps = int(t_m / dt)

    for t in range(time_steps):
        # Добавляем тепловыделение
        for i in range(len(source_positions)):
            x, y = source_positions[i]
            T[x, y] += source_powers[i] * (np.sin(2 * np.pi * t * dt / source_periods[i]) + 1) * dt

        # Численное решение уравнения диффузии (схема явная)
        T_new = T.copy()
        for i in range(1, N - 1):
            for j in range(1, N - 1):
                T_new[i, j] = T[i, j] + D[i, j] * dt / dx**2 * (
                    T[i + 1, j] + T[i - 1, j] + T[i, j + 1] + T[i, j - 1] - 4 * T[i, j]
                )
        T = T_new

        # Записываем температуры в точках проб
        for k, (x, y) in enumerate(probes):
            probe_temps[k, t] = T[x, y]

    return T, probe_temps

# Основной код
N_values = [60, 120, 240, 480]
t_m = 600  # общее время моделирования
results = {}

for N in N_values:
    dt = 0.1 if N <= 120 else 0.05
    start_time = time.time()

    # Создание карты диффузии
    D = create_diffusion_matrix(N)

    # Моделирование
    T_final, probe_temps = heat_diffusion_optimized(N, t_m, dt, D, source_positions, source_powers, source_periods, probes)

    # Время расчета
    elapsed_time = time.time() - start_time
    print(f'N={N}, Время расчета: {elapsed_time:.2f} секунд')

    # Сохранение результатов
    np.save(f'temperature_field_N{N}.npy', T_final)
    np.save(f'probe_temps_N{N}.npy', probe_temps)
    results[N] = (T_final, probe_temps)

    # Построение карты температур
    plt.figure(figsize=(8, 6))
    plt.imshow(T_final, origin='lower', cmap='hot')
    plt.colorbar(label='Температура')
    plt.scatter(*zip(*probes), c='black', marker='o')
    for i, (x, y) in enumerate(probes):
        plt.text(y, x, str(i), color='white', fontsize=12)
    plt.title(f'Поле температур при N={N}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.savefig(f'temperature_field_N{N}.png')
    plt.close()

    # Построение карты коэффициентов диффузии
    plt.figure(figsize=(8, 6))
    plt.imshow(D, origin='lower', cmap='coolwarm')
    plt.colorbar(label='Коэффициент диффузии')
    plt.title(f'Коэффициенты диффузии при N={N}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.savefig(f'diffusion_coeffs_N{N}.png')
    plt.close()

# Построение графиков температур в точках проб
plt.figure(figsize=(10, 6))
time_points = np.arange(0, t_m, dt)
for i, temps in enumerate(probe_temps):
    plt.plot(time_points, temps, label=f'Проба {i}')
plt.xlabel('Время (с)')
plt.ylabel('Температура')
plt.legend()
plt.title('Температура в точках проб')
plt.savefig('probe_temperatures.png')
plt.close()

# Анализ периодов с использованием БПФ
plt.figure(figsize=(10, 6))
for i, temps in enumerate(probe_temps):
    fft_vals = np.fft.rfft(temps)
    freqs = np.fft.rfftfreq(len(temps), dt)
    peaks, _ = find_peaks(np.abs(fft_vals))
    plt.plot(freqs, np.abs(fft_vals), label=f'Проба {i}')
plt.xlabel('Частота (Гц)')
plt.ylabel('Амплитуда')
plt.legend()
plt.title('Спектр температурных колебаний')
plt.savefig('fft_probes.png')
plt.close()


N=60, Время расчета: 8.15 секунд
N=120, Время расчета: 0.20 секунд
N=240, Время расчета: 1.02 секунд
N=480, Время расчета: 6.92 секунд
